In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [2]:
#get root folder
dir_working = os.getcwd()
print(dir_working)

#define folders
dir_process  = os.path.join(dir_working, "intermediate")
dir_results  = os.path.join(dir_working, "results"     )
dir_inputs   = os.path.join(dir_working, "input"       )
dir_data     = os.path.join(dir_inputs , "REMMdata"    )

AnalysisAreasInput = os.path.join(dir_inputs, r"residential_classes.gdb\residential_classes")
BaseParcels        = os.path.join(dir_data  , r"parcels\Center01102019.gdb\BaseGP01102019"  )
CenterParcels      = os.path.join(dir_data  , r"parcels\Center01102019.gdb\Center01102019"  )
Buildings          = os.path.join(dir_data  , r"BASE_run_285_2015_parcel_dev.csv"           )
ClassParameters    = os.path.join(dir_inputs, r"class_parameters.csv"                       )

ProcessGDB = "process.gdb"

#name of new data features
AnalysisAreas         = os.path.join(dir_process, ProcessGDB + "\AnalysisAreas"        )
AnalysisAreas_ByName  = os.path.join(dir_process, ProcessGDB + "\AnalysisAreas_ByName" )
AnalysisAreas_ByClass = os.path.join(dir_process, ProcessGDB + "\AnalysisAreas_ByClass")
AnalysisAreas_IDs     = os.path.join(dir_process, ProcessGDB + "\AnalysisAreas_IDs"    )
Parcels               = os.path.join(dir_process, ProcessGDB + "\Parcels"              )
ParcelsAA_join        = os.path.join(dir_process, ProcessGDB + "\ParcelsAA_join"       )
ParcelsAA             = os.path.join(dir_process, ProcessGDB + "\ParcelsAA"            )

#name of new data features
AnalysisAreas_shp         = os.path.join(dir_process, "AnalysisAreas.shp"        )
AnalysisAreas_ByName_shp  = os.path.join(dir_process, "AnalysisAreas_ByName.shp" )
AnalysisAreas_ByClass_shp = os.path.join(dir_process, "AnalysisAreas_ByClass.shp")
AnalysisAreas_IDs_shp     = os.path.join(dir_process, "AnalysisAreas_IDs.shp"    )
Parcels_shp               = os.path.join(dir_process, "Parcels.shp"              )
ParcelsAA_shp             = os.path.join(dir_process, "ParcelsAA.shp"            )

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)

print(AnalysisAreasInput)

E:\GitHub\Residential-Capacity
E:\GitHub\Residential-Capacity\input\residential_classes.gdb\residential_classes


# Prep Analysis Areas

In [3]:
#Create Analysis Area Layer in Intermediate Files and Fill with a AAName that combines Name and TOD status

sAreaBlankName  = 'No Area Name'

sAAName  = "AnalysisAreaName" #field for calculated Analysis Area Name
sAAClass = "AnalysisAreaClass" #field for calculated Analysis Area Class

#if processing geodatabase doesn't exist, create it
print("Checking if " + ProcessGDB + " exists...")
if not arcpy.Exists(os.path.join(dir_process, ProcessGDB)):
    print("Creating " + ProcessGDB + "...")
    arcpy.management.CreateFileGDB(dir_process, ProcessGDB)
else:
    print(ProcessGDB + " exists...")

print ("Creating Analysis Input Area intermediate layer in " + ProcessGDB + "...")
deleteIfExists(AnalysisAreas)
arcpy.management.Copy(AnalysisAreasInput, AnalysisAreas)

## Subroutines

print ("Adding " + sAAName + " field to " + AnalysisAreas + "...")
arcpy.AddField_management(AnalysisAreas, sAAName , "TEXT", 50)
arcpy.AddField_management(AnalysisAreas, sAAClass, "TEXT", 50)

##create calculated name for analysis areas combining center name and tod status
#areaname_codeblock = """def createAreaName(centername, tod, objid):
#    if (centername!=''):
#        return centername
#    elif (centername=='' and tod == 'Yes'):
#        return '_TOD ' + str(objid)
#    elif (centername==''):
#        return 'No Area Name'"""

##create calculated Class for analysis area combining center Class and tod status
#centerClass_codeblock = """def createClassName(centerClass, tod):
#    if (centerClass!='' and tod == 'Yes'):
#        return centerClass + ' and TOD'
#    elif (centerClass=='' and tod == 'Yes'):
#        return 'TOD Only'
#    elif (centerClass!='' and tod == 'No'):
#        return centerClass + ' and No TOD'
#    elif (centerClass=='' and tod == 'No'):
#        return 'No Class No TOD'"""

areaname_codeblock = """def createAreaName(centername):
    return centername"""

#create calculated Class for analysis area combining center Class and tod status
centerClass_codeblock = """def createClassName(centerClass):
   return centerClass"""


print ("Calculating " + sAAName + " field...")
arcpy.CalculateField_management(AnalysisAreas,
                                sAAName,
                                "createAreaName(!city!)",
                                "PYTHON_9.3",
                                areaname_codeblock)
print ("Calculating " + sAAClass + " field...")
arcpy.CalculateField_management(AnalysisAreas,
                                sAAClass,
                                "createClassName(!code!)",
                                "PYTHON_9.3",
                                centerClass_codeblock)

print ("Creating " + AnalysisAreas_ByName + " intermediate layer in " + ProcessGDB + "...")
deleteIfExists(AnalysisAreas_ByName)
arcpy.management.Dissolve(AnalysisAreas, AnalysisAreas_ByName,  sAAName)


print ("Creating " + AnalysisAreas_ByClass + " intermediate layer in " + ProcessGDB + "...")
deleteIfExists(AnalysisAreas_ByClass)
arcpy.management.Dissolve(AnalysisAreas, AnalysisAreas_ByClass, sAAClass)


print ("All Done")

Checking if process.gdb exists...
process.gdb exists...
Creating Analysis Input Area intermediate layer in process.gdb...
Adding AnalysisAreaName field to E:\GitHub\Residential-Capacity\intermediate\process.gdb\AnalysisAreas...
Calculating AnalysisAreaName field...
Calculating AnalysisAreaClass field...
Creating E:\GitHub\Residential-Capacity\intermediate\process.gdb\AnalysisAreas_ByName intermediate layer in process.gdb...
Creating E:\GitHub\Residential-Capacity\intermediate\process.gdb\AnalysisAreas_ByClass intermediate layer in process.gdb...
All Done


In [4]:
#create csv shells for values - NOT NEEDED IF IDS AND CLASSES ALREADY SET UP IN PARAMETERS

#Areas data
df_AnalysisAreas = pd.DataFrame()

cursorName = arcpy.SearchCursor(AnalysisAreas_ByName)
row = cursorName.next()
while row:
    #print(row.getValue(sAAName))
    df_AnalysisAreas = df_AnalysisAreas.append({sAAName : row.getValue(sAAName)},ignore_index=True)
    row = cursorName.next()  
    
df_AnalysisAreas.index.name = 'AreaID'
df_AnalysisAreas = df_AnalysisAreas.reset_index()
df_AnalysisAreas.to_csv(os.path.join(dir_results, r'areas.csv'),index=False)
display(df_AnalysisAreas)


#Classes data
df_Classes = pd.DataFrame()

cursorClass = arcpy.SearchCursor(AnalysisAreas_ByClass)
row = cursorClass.next()
while row:
    #print(row.getValue(sAAClass))
    df_Classes = df_Classes.append({'ClassDescription' : row.getValue(sAAClass)},ignore_index=True)
    row = cursorClass.next()

lColumns_Class = ['SFSplitRes','SFSplitCom','CapacityRes_DUA','CapacityCom_FAR','SFperHH','SFperEmp','PercentOpenSpace','RedevValuePerAcreRes','RedevValuePerAcreCom','RedevAndOr','RedevBldgAgeRes','RedevBldgAgeCom','GrowthRateRes','GrowthRateCom','RedevProb','ClassOrder']

for col in lColumns_Class:
    df_Classes[col] = ""
    
df_Classes.index.name = 'ClassID'
    
display(df_Classes)

df_Classes.to_csv(os.path.join(dir_process, r'class_parameters_shell.csv'))


,AreaID,AnalysisAreaName
0,0,None
1,1,Alpine
2,2,Alta
3,3,American Fork
4,4,Bluffdale
...,...,...
73,73,West Point
74,74,West Valley City
75,75,White City
76,76,Woodland Hills


,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes,RedevBldgAgeCom,GrowthRateRes,GrowthRateCom,RedevProb,ClassOrder
ClassID,,,,,,,,,,,,,,,,,
0,a1,,,,,,,,,,,,,,,,
1,a2,,,,,,,,,,,,,,,,
2,a3,,,,,,,,,,,,,,,,
3,a4,,,,,,,,,,,,,,,,
4,a5,,,,,,,,,,,,,,,,
5,b1,,,,,,,,,,,,,,,,
6,b2,,,,,,,,,,,,,,,,
7,b3,,,,,,,,,,,,,,,,
8,b4,,,,,,,,,,,,,,,,


In [5]:
df_ClassParam = pd.read_csv(ClassParameters)
display(df_ClassParam)

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder
0,f4,Multifamily Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,29
1,f3,Multifamily City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,28
2,f2,Multifamily Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,27
3,f1,Multifamily Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,No,26
4,g4,Mixed-Use Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,33
5,g3,Mixed-Use City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,32
6,g2,Mixed-Use Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31
7,g1,Mixed-Use Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,No,30
8,h4,Mixed Residential Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,37
9,h3,Mixed Residential City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,36


In [6]:
#create layer with just IDs to make union cleaner

df_ClassIDs = df_ClassParam[['ClassID','ClassDescription']]
#display(df_ClassIDs)

sdf_AA = pd.DataFrame.spatial.from_featureclass(AnalysisAreas)
#display(sdf_AnalysisAreas)

sdf_AAwClassID =  pd.DataFrame.merge(sdf_AA,df_ClassIDs,left_on=sAAClass, right_on='ClassID')
#display(sdf_AAwID)

sdf_AAwClassIDwNameID =  pd.DataFrame.merge(sdf_AAwClassID,df_AnalysisAreas,on=sAAName)
#display(sdf_AAwClassIDwNameID)

sdf_AAIDs = sdf_AAwClassIDwNameID[['AreaID','ClassID','SHAPE']]
#display(sdf_AAIDs)

#arcpy.Delete_management(AnalysisAreas_IDs)
deleteIfExists(AnalysisAreas_IDs)
sdf_AAIDs.spatial.to_featureclass(location=AnalysisAreas_IDs)
print('All Done')

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\geo\_accessor.py:1169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._data[col] = GeoArray(self._data[col])


All Done


In [7]:
sdf_AAwClassIDwNameID

,OBJECTID,city,county,city_lu_type,gen_lu_type,max_dua,code,plan_year,plan_source,data_source,AnalysisAreaName,AnalysisAreaClass,SHAPE,ClassID,ClassDescription,AreaID
0,1,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,None,None,Bluffdale,a4,"{'rings': [[[420271.3705000002, 4480517.338500...",a4,Single Family A4,4
1,3,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,None,None,Bluffdale,a4,"{'rings': [[[420636.9475999996, 4481132.772299...",a4,Single Family A4,4
2,4,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,None,None,Bluffdale,a4,"{'rings': [[[420671.3481999999, 4481261.932600...",a4,Single Family A4,4
3,6,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,None,None,Bluffdale,a4,"{'rings': [[[420869.50710000005, 4482955.5603]...",a4,Single Family A4,4
4,20,Bluffdale,Salt Lake,Very Low Density Residential,Residential SF,0.5,a4,2019.0,None,None,Bluffdale,a4,"{'rings': [[[421226.04870000016, 4479294.0206]...",a4,Single Family A4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11136,8168,Clearfield,Davis,Mixed Use,Mixed Use,30.0,g3,2017.0,https://clearfield.city/wp-content/uploads/202...,https://clearfield.maps.arcgis.com/apps/webapp...,Clearfield,g3,"{'rings': [[[414942.4210000001, 4550112.225299...",g3,Mixed-Use City/TOD,10
11137,8169,Clearfield,Davis,Mixed Use,Mixed Use,30.0,g3,2017.0,https://clearfield.city/wp-content/uploads/202...,https://clearfield.maps.arcgis.com/apps/webapp...,Clearfield,g3,"{'rings': [[[417057.5292999996, 4550162.428400...",g3,Mixed-Use City/TOD,10
11138,8170,Clearfield,Davis,Mixed Use,Mixed Use,30.0,g3,2017.0,https://clearfield.city/wp-content/uploads/202...,https://clearfield.maps.arcgis.com/apps/webapp...,Clearfield,g3,"{'rings': [[[416772.02419999987, 4550546.96120...",g3,Mixed-Use City/TOD,10
11139,8171,Clearfield,Davis,Mixed Use,Mixed Use,30.0,g3,2017.0,https://clearfield.city/wp-content/uploads/202...,https://clearfield.maps.arcgis.com/apps/webapp...,Clearfield,g3,"{'rings': [[[414009.90950000007, 4551759.66699...",g3,Mixed-Use City/TOD,10


# Prep Parcels

In [8]:
#read in parcels
sdf_bPar = pd.DataFrame.spatial.from_featureclass(BaseParcels)

In [9]:
sdf_bPar

,OBJECTID,parcel_id,parcel_id_REMM,basebldg,Sliver,parcel_sqft,Split,x,y,parcel_acre,...,type5,type6,type7,type8,max_height,Agriculture,NoBuild,redev_friction,note,SHAPE
0,1,671124.0,671124.0,0.0,None,16203.648534,None,1.572705e+06,7.294321e+06,0.371986,...,t,f,f,f,999.0,0.0,NaN,NaN,None,"{""rings"": [[[1572658.531162396, 7294412.922931..."
1,2,588354.0,588354.0,0.0,None,49466.799191,None,1.572481e+06,7.293816e+06,1.135606,...,t,f,f,f,999.0,0.0,NaN,NaN,None,"{""rings"": [[[1572446.1962272525, 7294150.73555..."
2,3,664267.0,664267.0,1.0,None,27944.879802,None,1.571343e+06,7.291668e+06,0.641529,...,f,f,f,f,999.0,0.0,NaN,NaN,None,"{""rings"": [[[1571352.9701225907, 7291780.92202..."
3,4,632453.0,632453.0,1.0,None,15190.031848,None,1.580054e+06,7.303541e+06,0.348717,...,f,f,f,f,999.0,0.0,NaN,NaN,None,"{""rings"": [[[1579974.9191654772, 7303604.60588..."
4,5,632450.0,632450.0,1.0,None,10092.376514,None,1.580190e+06,7.303272e+06,0.231690,...,f,f,f,f,999.0,0.0,NaN,NaN,None,"{""rings"": [[[1580242.158429563, 7303338.493389..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845215,845216,601655.0,886696.0,0.0,None,137565.519929,1,1.573685e+06,7.220955e+06,3.158082,...,f,f,f,f,999.0,1.0,NaN,None,None,"{""rings"": [[[1573850.763353318, 7220743.714214..."
845216,845217,601655.0,886697.0,0.0,None,213243.313865,1,1.573302e+06,7.220958e+06,4.895412,...,f,f,f,f,999.0,1.0,NaN,None,None,"{""rings"": [[[1573537.6948660016, 7220743.71421..."
845217,845218,601655.0,886698.0,0.0,None,214262.868997,1,1.572836e+06,7.220958e+06,4.918818,...,f,f,f,f,999.0,1.0,NaN,None,None,"{""rings"": [[[1573072.201039672, 7220743.714214..."
845218,845219,601655.0,886699.0,0.0,None,55026.868822,1,1.572074e+06,7.220958e+06,1.263248,...,f,f,f,f,999.0,0.0,NaN,None,None,"{""rings"": [[[1572137.6125155538, 7220743.71421..."


In [10]:
#set field for commercial, residential, or mixed

sdf_bPar['luType'] = ''

#undevelopable land (all building types are 'f')
sdf_bPar.loc[((sdf_bPar['type1'] == 'f') &
              (sdf_bPar['type2'] == 'f') &
              (sdf_bPar['type3'] == 'f') &
              (sdf_bPar['type4'] == 'f') &
              (sdf_bPar['type5'] == 'f') &
              (sdf_bPar['type6'] == 'f') &
              (sdf_bPar['type7'] == 'f') &
              (sdf_bPar['type8'] == 'f')), 'luType'] = 'Undevelopable'

sdf_bPar = sdf_bPar.drop(columns=['OBJECTID'      ,
                                  'city'          ,
                                  'x'             ,
                                  'y'             ,
                                  'county_name'   ,
                                  'distsml_id'    ,
                                  'distmed_id'    ,
                                  'distlrg_id'    ,
                                  'county_taz_id' ,
                                  'NoBuild'       ,
                                  'redev_friction',
                                  'Agriculture'   ,
                                  'Sliver'        ,
                                  'Split'         ,
                                  'utmxi'         ,
                                  'utmyi'         ,
                                  'note'          ,
                                  'zone_id'       ,
                                  'type1'         ,
                                  'type2'         ,
                                  'type3'         ,
                                  'type4'         ,
                                  'type5'         ,
                                  'type6'         ,
                                  'type7'         ,
                                  'type8'])

In [11]:
sdf_bPar

,parcel_id,parcel_id_REMM,basebldg,parcel_sqft,parcel_acre,county_id,max_dua,max_far,max_height,SHAPE,luType
0,671124.0,671124.0,0.0,16203.648534,0.371986,4.0,NaN,0.50,999.0,"{""rings"": [[[1572658.531162396, 7294412.922931...",
1,588354.0,588354.0,0.0,49466.799191,1.135606,4.0,NaN,0.50,999.0,"{""rings"": [[[1572446.1962272525, 7294150.73555...",
2,664267.0,664267.0,1.0,27944.879802,0.641529,4.0,NaN,0.41,999.0,"{""rings"": [[[1571352.9701225907, 7291780.92202...",
3,632453.0,632453.0,1.0,15190.031848,0.348717,4.0,4.000,NaN,999.0,"{""rings"": [[[1579974.9191654772, 7303604.60588...",
4,632450.0,632450.0,1.0,10092.376514,0.231690,4.0,4.000,NaN,999.0,"{""rings"": [[[1580242.158429563, 7303338.493389...",
...,...,...,...,...,...,...,...,...,...,...,...
845215,601655.0,886696.0,0.0,137565.519929,3.158082,4.0,0.025,NaN,999.0,"{""rings"": [[[1573850.763353318, 7220743.714214...",
845216,601655.0,886697.0,0.0,213243.313865,4.895412,4.0,0.025,NaN,999.0,"{""rings"": [[[1573537.6948660016, 7220743.71421...",
845217,601655.0,886698.0,0.0,214262.868997,4.918818,4.0,0.025,NaN,999.0,"{""rings"": [[[1573072.201039672, 7220743.714214...",
845218,601655.0,886699.0,0.0,55026.868822,1.263248,4.0,0.025,NaN,999.0,"{""rings"": [[[1572137.6125155538, 7220743.71421...",


In [12]:
sdf_bPar_luTypeEmpty = sdf_bPar.loc[(sdf_bPar['luType'] == '')]
sdf_bPar_luTypeEmpty

,parcel_id,parcel_id_REMM,basebldg,parcel_sqft,parcel_acre,county_id,max_dua,max_far,max_height,SHAPE,luType
0,671124.0,671124.0,0.0,16203.648534,0.371986,4.0,NaN,0.50,999.0,"{""rings"": [[[1572658.531162396, 7294412.922931...",
1,588354.0,588354.0,0.0,49466.799191,1.135606,4.0,NaN,0.50,999.0,"{""rings"": [[[1572446.1962272525, 7294150.73555...",
2,664267.0,664267.0,1.0,27944.879802,0.641529,4.0,NaN,0.41,999.0,"{""rings"": [[[1571352.9701225907, 7291780.92202...",
3,632453.0,632453.0,1.0,15190.031848,0.348717,4.0,4.000,NaN,999.0,"{""rings"": [[[1579974.9191654772, 7303604.60588...",
4,632450.0,632450.0,1.0,10092.376514,0.231690,4.0,4.000,NaN,999.0,"{""rings"": [[[1580242.158429563, 7303338.493389...",
...,...,...,...,...,...,...,...,...,...,...,...
845215,601655.0,886696.0,0.0,137565.519929,3.158082,4.0,0.025,NaN,999.0,"{""rings"": [[[1573850.763353318, 7220743.714214...",
845216,601655.0,886697.0,0.0,213243.313865,4.895412,4.0,0.025,NaN,999.0,"{""rings"": [[[1573537.6948660016, 7220743.71421...",
845217,601655.0,886698.0,0.0,214262.868997,4.918818,4.0,0.025,NaN,999.0,"{""rings"": [[[1573072.201039672, 7220743.714214...",
845218,601655.0,886699.0,0.0,55026.868822,1.263248,4.0,0.025,NaN,999.0,"{""rings"": [[[1572137.6125155538, 7220743.71421...",


In [13]:
sdf_bPar_lutypeTotals = sdf_bPar.groupby(['luType']).agg({"parcel_id": [np.size],"parcel_acre": [np.sum]})
sdf_bPar_lutypeTotals

,parcel_id,parcel_acre
,size,sum
luType,,
,769736.0,497362.532526
Undevelopable,75484.0,284200.922445


In [14]:
sdf_bParProcessed = sdf_bPar[['parcel_id_REMM','parcel_acre','luType','SHAPE']]

In [15]:
sdf_bParProcessed

,parcel_id_REMM,parcel_acre,luType,SHAPE
0,671124.0,0.371986,,"{""rings"": [[[1572658.531162396, 7294412.922931..."
1,588354.0,1.135606,,"{""rings"": [[[1572446.1962272525, 7294150.73555..."
2,664267.0,0.641529,,"{""rings"": [[[1571352.9701225907, 7291780.92202..."
3,632453.0,0.348717,,"{""rings"": [[[1579974.9191654772, 7303604.60588..."
4,632450.0,0.231690,,"{""rings"": [[[1580242.158429563, 7303338.493389..."
...,...,...,...,...
845215,886696.0,3.158082,,"{""rings"": [[[1573850.763353318, 7220743.714214..."
845216,886697.0,4.895412,,"{""rings"": [[[1573537.6948660016, 7220743.71421..."
845217,886698.0,4.918818,,"{""rings"": [[[1573072.201039672, 7220743.714214..."
845218,886699.0,1.263248,,"{""rings"": [[[1572137.6125155538, 7220743.71421..."


In [16]:
#read in parcels with Center data - MAX DUA and MAX FAR
sdf_cPar = pd.DataFrame.spatial.from_featureclass(CenterParcels)

In [17]:
sdf_cPar

,OBJECTID,parcel_id_REMM,max_dua,max_far,year,type1,type2,type3,type4,type5,type6,type7,type8,note,NoBuild,PolicyKey,SHAPE
0,1,671124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,"{""rings"": [[[1572658.531162396, 7294412.922931..."
1,2,588354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,"{""rings"": [[[1572446.1962272525, 7294150.73555..."
2,3,664267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,"{""rings"": [[[1571352.9701225907, 7291780.92202..."
3,4,632453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,"{""rings"": [[[1579974.9191654772, 7303604.60588..."
4,5,632450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,"{""rings"": [[[1580242.158429563, 7303338.493389..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845214,845215,886695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,"{""rings"": [[[1574478.0519851, 7220743.71421465..."
845215,845216,886696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,"{""rings"": [[[1573850.763353318, 7220743.714214..."
845216,845217,886697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,"{""rings"": [[[1573537.6948660016, 7220743.71421..."
845217,845218,886698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,"{""rings"": [[[1573072.201039672, 7220743.714214..."


In [18]:
df_cParMax = sdf_cPar.groupby(['parcel_id_REMM'], as_index=False).agg({"max_dua": [np.max],"max_far": [np.max]})

#drop the second header row to allow ease of calcs
df_cParMax.columns = df_cParMax.columns.droplevel(1)

df_cParMax = df_cParMax.dropna(thresh=2) #drop rows with at least 2 NaNs

df_cParMax.columns = ('parcel_id_REMM','max_dua_center','max_far_center')
df_cParMax

,parcel_id_REMM,max_dua_center,max_far_center
5,7,25.0,1.00
28,43,0.0,0.41
332,486,0.0,0.41
376,533,30.0,2.00
548,749,0.0,0.41
...,...,...,...
842926,884406,0.0,0.41
842927,884407,0.0,0.41
842928,884408,0.0,0.41
842929,884409,0.0,0.41


In [19]:

df_parcelwithbldg = pd.read_csv(Buildings)


In [20]:
display(Buildings)

'E:\\GitHub\\Residential-Capacity\\input\\REMMdata\\BASE_run_285_2015_parcel_dev.csv'

In [21]:
pd.set_option('display.max_columns', None)
df_parcelwithbldg

,parcel_id,parcel_id_REMM,county_id,zone_id,parcel_acres,land_value,max_far,max_dua,type1,type2,type3,type4,type5,type6,type7,type8,has_buildings,was_developed,was_redeveloped,developable,residential_units,job_spaces,building_sqft,non_residential_sqft,residential_sqft,unit_price_non_residential,res_price_per_sqft,is_sf,is_mf,is_industrial,is_retail,is_office,is_govt,is_mixeduse,is_other,year_built,building_count,non_res_value,res_value,total_value
0,741871,741871,49,2111,0.197968,35700.00,NaN,6.000,t,f,f,f,f,f,f,f,1,0,0,1.0,1.0,0.0,2434.0,0.0,2434.0,0.0,74.185722,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2005.0,1.0,0.0,180568.046472,216268.046472
1,579822,579822,49,2873,37.717834,56614.32,NaN,NaN,f,f,f,f,f,f,f,f,0,0,0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,56614.320000
2,579853,579853,49,2117,6.791235,81500.00,NaN,6.000,t,f,f,f,f,f,f,f,0,0,0,1.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,81500.000000
3,640185,640185,49,2073,1.372797,112100.00,NaN,2.500,t,f,f,f,f,f,f,f,1,0,0,1.0,1.0,0.0,5499.0,0.0,5499.0,0.0,45.462811,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1981.0,1.0,0.0,249999.999999,362099.999999
4,682698,682698,49,2119,0.163751,43000.00,NaN,0.200,t,f,f,f,f,f,f,f,1,0,0,1.0,1.0,0.0,3046.0,0.0,3046.0,0.0,61.906857,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2006.0,1.0,0.0,188568.285417,231568.285417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841679,886695,886695,49,2666,4.531522,48006.00,NaN,0.025,t,f,f,f,f,f,f,f,0,0,0,1.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,48006.000000
841680,886696,886696,49,2666,3.158082,33456.00,NaN,0.025,t,f,f,f,f,f,f,f,0,0,0,1.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,33456.000000
841681,886697,886697,49,2666,4.895412,51862.00,NaN,0.025,t,f,f,f,f,f,f,f,0,0,0,1.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,51862.000000
841682,886698,886698,49,2666,4.918818,52109.00,NaN,0.025,t,f,f,f,f,f,f,f,0,0,0,1.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,52109.000000


In [22]:
df_bldg = df_parcelwithbldg[df_parcelwithbldg['has_buildings']==1].copy()

df_bldg['bldgtype'] = ''

#residential building types
df_bldg.loc[(df_bldg['is_sf'] != 0) | (df_bldg['is_mf'] != 0), 'bldgtype'] = 'Residential'

#commercial building types, if mixed use (res and com) then use commercial 
df_bldg.loc[((df_bldg['is_industrial'] != 0) |
             (df_bldg['is_retail'    ] != 0) |
             (df_bldg['is_office'    ] != 0) |
             (df_bldg['is_govt'      ] != 0) |
             (df_bldg['is_mixeduse'  ] != 0) |
             (df_bldg['is_other'     ] != 0)), 'bldgtype'] = 'Commercial'

df_bldg

,parcel_id,parcel_id_REMM,county_id,zone_id,parcel_acres,land_value,max_far,max_dua,type1,type2,type3,type4,type5,type6,type7,type8,has_buildings,was_developed,was_redeveloped,developable,residential_units,job_spaces,building_sqft,non_residential_sqft,residential_sqft,unit_price_non_residential,res_price_per_sqft,is_sf,is_mf,is_industrial,is_retail,is_office,is_govt,is_mixeduse,is_other,year_built,building_count,non_res_value,res_value,total_value,bldgtype
0,741871,741871,49,2111,0.197968,35700.0,NaN,6.000,t,f,f,f,f,f,f,f,1,0,0,1.0,1.0,0.0,2434.0,0.0,2434.0,0.0,74.185722,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2005.0,1.0,0.0,180568.046472,2.162680e+05,Residential
3,640185,640185,49,2073,1.372797,112100.0,NaN,2.500,t,f,f,f,f,f,f,f,1,0,0,1.0,1.0,0.0,5499.0,0.0,5499.0,0.0,45.462811,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1981.0,1.0,0.0,249999.999999,3.621000e+05,Residential
4,682698,682698,49,2119,0.163751,43000.0,NaN,0.200,t,f,f,f,f,f,f,f,1,0,0,1.0,1.0,0.0,3046.0,0.0,3046.0,0.0,61.906857,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2006.0,1.0,0.0,188568.285417,2.315683e+05,Residential
6,693553,693553,49,2147,0.487942,87700.0,NaN,3.000,t,f,f,f,f,f,f,f,1,0,0,1.0,1.0,0.0,3268.0,0.0,3268.0,0.0,56.303550,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1979.0,1.0,0.0,184000.000027,2.717000e+05,Residential
7,658088,658088,49,2143,0.337967,76200.0,NaN,NaN,f,f,f,f,f,f,f,f,1,0,0,0.0,1.0,0.0,4408.0,0.0,4408.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2006.0,1.0,0.0,0.000000,7.620000e+04,Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841528,601446,601446,49,2651,2.902050,14492.0,NaN,NaN,f,f,f,f,f,f,f,f,1,0,0,0.0,1.0,0.0,2935.0,0.0,2935.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1997.0,1.0,0.0,0.000000,1.449200e+04,Residential
841575,609702,609702,49,2731,0.515341,7146.0,NaN,6.000,t,f,f,f,f,f,f,f,1,0,0,1.0,1.0,0.0,2958.0,0.0,2958.0,0.0,36.308316,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1956.0,1.0,0.0,107400.000000,1.145460e+05,Residential
841587,603550,603550,49,2687,4.903986,60519.0,NaN,0.200,t,f,f,f,f,f,f,f,1,0,0,1.0,1.0,0.0,1296.0,0.0,1296.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1971.0,1.0,0.0,0.000000,6.051900e+04,Residential
841621,581782,581782,49,2069,35.460532,1261202.0,NaN,2.500,t,f,f,f,f,f,f,f,1,0,0,1.0,1.0,0.0,1414.0,0.0,1414.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1917.0,1.0,0.0,0.000000,1.261202e+06,Residential


In [23]:
df_bldg[df_bldg['parcel_id']==77033]

,parcel_id,parcel_id_REMM,county_id,zone_id,parcel_acres,land_value,max_far,max_dua,type1,type2,type3,type4,type5,type6,type7,type8,has_buildings,was_developed,was_redeveloped,developable,residential_units,job_spaces,building_sqft,non_residential_sqft,residential_sqft,unit_price_non_residential,res_price_per_sqft,is_sf,is_mf,is_industrial,is_retail,is_office,is_govt,is_mixeduse,is_other,year_built,building_count,non_res_value,res_value,total_value,bldgtype
443785,77033,77033,35,969,6.136467,4282341.82,9.37,125.0,t,t,f,t,t,f,f,f,1,0,0,1.0,144.0,0.0,161779.0,0.0,161779.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2011.0,144.0,0.0,0.0,4282341.82,Residential


In [24]:
#"bldgtype":[np.min] --> 'Commercial' < 'Residential', pick commercial if a mix of types

df_bldg_summary = df_bldg.groupby(['parcel_id'], as_index=False).agg({"bldgtype":[np.min], "year_built":[np.max], "residential_units":[np.sum], "job_spaces":[np.sum], "residential_sqft":[np.sum], "non_residential_sqft":[np.sum], "res_price_per_sqft":[np.mean], "unit_price_non_residential":[np.mean]})
df_bldg_summary.columns = df_bldg_summary.columns.droplevel(1)

df_bldg_summary['value_res'] = df_bldg_summary['residential_sqft'] * df_bldg_summary['res_price_per_sqft']
df_bldg_summary['value_com'] = df_bldg_summary['non_residential_sqft'] * df_bldg_summary['unit_price_non_residential']

df_bldg_summary.columns = ('parcel_id','bldgtype','yearbuilt','resunits','job_spaces','sf_res','sf_com','sfvalue_res','sfvalue_com','value_res','value_com')

df_bldg_summary.to_csv(os.path.join(dir_process, "bldgsummary.csv"))
df_bldg_summary

,parcel_id,bldgtype,yearbuilt,resunits,job_spaces,sf_res,sf_com,sfvalue_res,sfvalue_com,value_res,value_com
0,44,Commercial,1952.0,0.0,39.0,0.0,14586.0,0.0,0.000000,0.0,0.000000
1,152,Residential,1951.0,1.0,0.0,2198.0,0.0,0.0,0.000000,0.0,0.000000
2,215,Commercial,1979.0,0.0,260.0,0.0,97240.0,0.0,0.000000,0.0,0.000000
3,278,Commercial,1914.0,0.0,12.0,0.0,4488.0,0.0,0.000000,0.0,0.000000
4,308,Commercial,1948.0,0.0,14.0,0.0,13650.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
615793,871288,Residential,2007.0,1.0,0.0,3269.0,0.0,0.0,0.000000,0.0,0.000000
615794,872441,Residential,1932.0,1.0,0.0,1680.0,0.0,0.0,0.000000,0.0,0.000000
615795,875208,Residential,2014.0,1.0,0.0,2971.0,0.0,0.0,0.000000,0.0,0.000000
615796,882712,Commercial,1996.0,0.0,1.0,0.0,350.0,0.0,30.428571,0.0,10650.000001


In [25]:
sdf_bPar_cParMax = pd.DataFrame.merge(sdf_bPar,df_cParMax,on='parcel_id_REMM',how='left')
sdf_bPar_cParMax_bldg = pd.DataFrame.merge(sdf_bPar_cParMax,df_bldg_summary,on='parcel_id',how='left')
sdf_bPar_cParMax_bldg

,parcel_id,parcel_id_REMM,basebldg,parcel_sqft,parcel_acre,county_id,max_dua,max_far,max_height,SHAPE,luType,max_dua_center,max_far_center,bldgtype,yearbuilt,resunits,job_spaces,sf_res,sf_com,sfvalue_res,sfvalue_com,value_res,value_com
0,671124.0,671124.0,0.0,16203.648534,0.371986,4.0,NaN,0.50,999.0,"{'rings': [[[1572658.531162396, 7294412.922931...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,588354.0,588354.0,0.0,49466.799191,1.135606,4.0,NaN,0.50,999.0,"{'rings': [[[1572446.1962272525, 7294150.73555...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,664267.0,664267.0,1.0,27944.879802,0.641529,4.0,NaN,0.41,999.0,"{'rings': [[[1571352.9701225907, 7291780.92202...",,NaN,NaN,Commercial,2010.0,0.0,4.0,0.0,3900.0,0.000000,0.0,0.000000,0.0
3,632453.0,632453.0,1.0,15190.031848,0.348717,4.0,4.000,NaN,999.0,"{'rings': [[[1579974.9191654772, 7303604.60588...",,NaN,NaN,Residential,1998.0,1.0,0.0,2194.0,0.0,78.133578,0.0,171425.070988,0.0
4,632450.0,632450.0,1.0,10092.376514,0.231690,4.0,4.000,NaN,999.0,"{'rings': [[[1580242.158429563, 7303338.493389...",,NaN,NaN,Residential,1997.0,1.0,0.0,1698.0,0.0,83.941064,0.0,142531.926978,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845215,601655.0,886696.0,0.0,137565.519929,3.158082,4.0,0.025,NaN,999.0,"{'rings': [[[1573850.763353318, 7220743.714214...",,NaN,NaN,Residential,1892.0,1.0,0.0,678.0,0.0,40.560472,0.0,27500.000002,0.0
845216,601655.0,886697.0,0.0,213243.313865,4.895412,4.0,0.025,NaN,999.0,"{'rings': [[[1573537.6948660016, 7220743.71421...",,NaN,NaN,Residential,1892.0,1.0,0.0,678.0,0.0,40.560472,0.0,27500.000002,0.0
845217,601655.0,886698.0,0.0,214262.868997,4.918818,4.0,0.025,NaN,999.0,"{'rings': [[[1573072.201039672, 7220743.714214...",,NaN,NaN,Residential,1892.0,1.0,0.0,678.0,0.0,40.560472,0.0,27500.000002,0.0
845218,601655.0,886699.0,0.0,55026.868822,1.263248,4.0,0.025,NaN,999.0,"{'rings': [[[1572137.6125155538, 7220743.71421...",,NaN,NaN,Residential,1892.0,1.0,0.0,678.0,0.0,40.560472,0.0,27500.000002,0.0


In [26]:
#sdf_bPar_cParMax_bldg.sum()

In [27]:
sdf_bPar_cParMax_bldg.spatial.to_featureclass(location=Parcels)

'E:\\GitHub\\Residential-Capacity\\intermediate\\process.gdb\\Parcels'

In [28]:
#clear memory

del sdf_bPar_cParMax_bldg
del sdf_bPar_cParMax
del sdf_bPar
del sdf_cPar
del df_bldg
del df_bldg_summary

# Intersect Analysis Areas with Parcels

In [29]:
print ("Calculating parcel acreage...")

# Execute DeleteField
arcpy.DeleteField_management(Parcels, ['Acres'])

arcpy.AddField_management(Parcels, "ParcelAcres" , "DOUBLE")
arcpy.CalculateField_management(Parcels,
                                "ParcelAcres",
                                "!shape.area@acres!",
                                "PYTHON_9.3")
print ("Done")

Calculating parcel acreage...
Done


In [30]:
deleteIfExists(ParcelsAA_join)

#intersection parcels with analysis area types
arcpy.analysis.Intersect([AnalysisAreas_IDs,Parcels], ParcelsAA_join)

<Result 'E:\\GitHub\\Residential-Capacity\\intermediate\\process.gdb\\ParcelsAA_join'>

In [31]:


print ("Calculating area...")
arcpy.AddField_management(ParcelsAA_join, "PieceAcres" , "DOUBLE")
arcpy.CalculateField_management(ParcelsAA_join,
                                "PieceAcres",
                                "!shape.area@acres!",
                                "PYTHON_9.3")
arcpy.AddField_management(ParcelsAA_join, "PiecePortion" , "DOUBLE")
arcpy.CalculateField_management(ParcelsAA_join,
                                "PiecePortion",
                                "$feature.PieceAcres / $feature.ParcelAcres", "ARCADE")

print ("Done")

Calculating area...
Done


In [32]:
sdf_ParcelsAA = pd.DataFrame.spatial.from_featureclass(ParcelsAA_join)
sdf_ParcelsAA.columns

Index(['OBJECTID', 'FID_AnalysisAreas_IDs', 'AreaID', 'ClassID', 'FID_Parcels',
       'parcel_id', 'parcel_id_REMM', 'basebldg', 'parcel_sqft', 'parcel_acre',
       'county_id', 'max_dua', 'max_far', 'max_height', 'luType',
       'max_dua_center', 'max_far_center', 'bldgtype', 'yearbuilt', 'resunits',
       'job_spaces', 'sf_res', 'sf_com', 'sfvalue_res', 'sfvalue_com',
       'value_res', 'value_com', 'ParcelAcres', 'PieceAcres', 'PiecePortion',
       'SHAPE'],
      dtype='object')

In [36]:
#adjust parcel values by portion of parcel in parcel piece
sdf_ParcelsAA['parcel_sqft'] = sdf_ParcelsAA['parcel_sqft'] * sdf_ParcelsAA['PiecePortion']
sdf_ParcelsAA['resunits'   ] = sdf_ParcelsAA['resunits'   ] * sdf_ParcelsAA['PiecePortion']
sdf_ParcelsAA['job_spaces' ] = sdf_ParcelsAA['job_spaces' ] * sdf_ParcelsAA['PiecePortion']
sdf_ParcelsAA['sf_res'     ] = sdf_ParcelsAA['sf_res'     ] * sdf_ParcelsAA['PiecePortion']
sdf_ParcelsAA['sf_com'     ] = sdf_ParcelsAA['sf_com'     ] * sdf_ParcelsAA['PiecePortion']
sdf_ParcelsAA['value_res'  ] = sdf_ParcelsAA['value_res'  ] * sdf_ParcelsAA['PiecePortion']
sdf_ParcelsAA['value_com'  ] = sdf_ParcelsAA['value_com'  ] * sdf_ParcelsAA['PiecePortion']

#make sure acres used in future is just for piece of parcel, not all
sdf_ParcelsAA['Acres'] = sdf_ParcelsAA['PieceAcres']
display(sdf_ParcelsAA)

display("exporting Parcels")

sdf_ParcelsAA.spatial.to_featureclass(location=ParcelsAA)

,OBJECTID,FID_AnalysisAreas_IDs,AreaID,ClassID,FID_Parcels,parcel_id,parcel_id_REMM,basebldg,parcel_sqft,parcel_acre,county_id,max_dua,max_far,max_height,luType,max_dua_center,max_far_center,bldgtype,yearbuilt,resunits,job_spaces,sf_res,sf_com,sfvalue_res,sfvalue_com,value_res,value_com,ParcelAcres,PieceAcres,PiecePortion,SHAPE,Acres
0,1,11110,18,a1,1912,579541.0,579541.0,0.0,1.416783e+05,3.255375,4.0,1.000000,NaN,999.0,,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.255373,3.253935e+00,9.995583e-01,"{""rings"": [[[407535.2560999999, 4457143.329299...",3.253935e+00
1,2,11110,18,a1,10577,579551.0,579551.0,1.0,4.321657e+04,0.997946,4.0,1.000000,NaN,999.0,,NaN,NaN,Residential,1976.0,9.941623e-01,0.000000e+00,1422.646275,0.000000,82.640882,0.000000,117568.743289,0.000000,0.997945,9.950284e-01,9.970769e-01,"{""rings"": [[[407213.32070000004, 4457172.3539]...",9.950284e-01
2,3,11110,18,a1,31345,579540.0,579540.0,0.0,1.416783e+05,3.255376,4.0,1.000000,NaN,999.0,,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.255376,3.253937e+00,9.995580e-01,"{""rings"": [[[407534.25019999966, 4456966.80010...",3.253937e+00
3,4,11110,18,a1,52886,579564.0,579564.0,0.0,8.686390e+04,1.995889,4.0,1.000000,NaN,999.0,,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.995888,1.995007e+00,9.995589e-01,"{""rings"": [[[407360.8212000001, 4457232.3221],...",1.995007e+00
4,5,11110,18,a1,90204,579574.0,579574.0,0.0,4.343180e+04,0.997944,4.0,1.000000,NaN,999.0,,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.997945,9.975029e-01,9.995573e-01,"{""rings"": [[[407591.59580000024, 4457228.08950...",9.975029e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939712,939713,46,5,c3,569725,444878.0,444878.0,0.0,1.082406e-10,6.878654,1.0,2.907546,NaN,999.0,,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.878651,1.307385e-07,1.900642e-08,"{""rings"": [[[428818.9419, 4527462.916099999], ...",1.307385e-07
939713,939714,46,5,c3,771763,370287.0,812860.0,0.0,1.475198e-08,0.050470,1.0,2.989457,NaN,999.0,,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.050471,1.307385e-07,2.590379e-06,"{""rings"": [[[428818.9419, 4527462.916099999], ...",1.307385e-07
939714,939715,47,5,c3,342265,356306.0,356306.0,1.0,6.440873e-04,0.401154,1.0,NaN,0.5,999.0,,NaN,NaN,Commercial,1989.0,0.000000e+00,3.317337e-07,0.000000,0.000210,0.000000,26.459265,0.000000,0.005556,0.401155,7.701683e-05,1.919878e-04,"{""rings"": [[[426694.8435000004, 4526659.646299...",7.701683e-05
939715,939716,47,5,c3,359251,408107.0,408107.0,1.0,6.980750e-04,0.370131,1.0,NaN,0.5,999.0,,NaN,NaN,Commercial,1990.0,0.000000e+00,2.164866e-07,0.000000,0.000137,0.000000,61.805978,0.000000,0.008470,0.370131,7.701683e-05,2.080801e-04,"{""rings"": [[[426694.8435000004, 4526659.646299...",7.701683e-05


'exporting Parcels'

'E:\\GitHub\\Residential-Capacity\\intermediate\\process.gdb\\ParcelsAA'

In [37]:
sdf_ParcelsAA.groupby(['AreaID','ClassID'], as_index=False).agg({"parcel_id": [np.size], "job_spaces": [np.sum], "resunits":[np.sum], "Acres":[np.sum]})

,AreaID,ClassID,parcel_id,job_spaces,resunits,Acres
,,,size,sum,sum,sum
0,0,a1,575.0,7.206084,93.060474,1554.873664
1,0,a2,4642.0,232.118444,1262.263010,7983.559693
2,0,a3,4597.0,292.415124,448.658337,11627.908363
3,0,a4,617.0,120.233696,18.102295,905.944246
4,0,a5,1203.0,267.243745,279.445881,2452.467821
...,...,...,...,...,...,...
647,76,a3,34.0,0.000000,0.000957,10.513478
648,77,d1,2842.0,108.960528,2521.094418,666.306789
649,77,d2,85.0,24.961343,67.445078,13.757016


In [38]:
#export to shp files for better drawing in Jupyter

deleteIfExists(AnalysisAreas_shp)
print ("Exporting " + AnalysisAreas_shp + "...")
arcpy.conversion.FeatureClassToShapefile(AnalysisAreas, dir_process)

deleteIfExists(AnalysisAreas_IDs_shp)
print ("Exporting " + AnalysisAreas_IDs_shp + "...")
arcpy.conversion.FeatureClassToShapefile(AnalysisAreas_IDs, dir_process)

deleteIfExists(ParcelsAA_shp)
print ("Exporting " + ParcelsAA_shp + "...")
arcpy.conversion.FeatureClassToShapefile(ParcelsAA, dir_process) 

display("done!")

Exporting E:\GitHub\Residential-Capacity\intermediate\AnalysisAreas.shp...
Exporting E:\GitHub\Residential-Capacity\intermediate\AnalysisAreas_IDs.shp...
Exporting E:\GitHub\Residential-Capacity\intermediate\ParcelsAA.shp...


<Result 'E:\\GitHub\\Residential-Capacity\\intermediate'>